### 문장분리를 kkma 대신 kss 사용함
### 문장 요약을 5로 설정하면 딱 5문장 나오도록 하기
### 짧은 기사는 대상에서 제외시키기 (크롤링할때 제외)
### 동영상 뉴스는 제외시키기(?)
## 크롤링 할 때 가중치 매기기 (컨텐츠 필터링)
## 유저 프로필 파악 (기사 랜덤으로 뽑아서 띄우기)

In [8]:
import urllib.request
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions

import kss

class newsCrawler:
    def __init__(self):
        self.titleList=[]
        self.contentsList=[]
        self.imageList=[]
        
    # 네이버 뉴스홈
    def mainCrawl(self):    
        # 정치=100 경제=101 사회=102 생활/문화=103 세계=104 IT/과학=105
        # 실제 실행에서는 range(100, 106)으로 해야함.
        for i in range(101, 102):
            main_url = "https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1="+str(i)
            driver.get(main_url)
            
            # '헤드라인 더보기' 버튼이 있다면 누르기       
            self.showMore()
            driver.implicitly_wait(3)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')  
            
            # 헤드라인 가져오기
            self.subCrawl(soup)
        driver.quit()
    
    # 더보기버튼 클릭
    def showMore(self):
        try:
            while True:
                driver.find_element_by_xpath('//*[@id="main_content"]/div/div[2]/div[2]/div/a').click()
        except exceptions.ElementNotVisibleException as e:
            pass
        except Exception as e:
            pass
    
    # 헤드라인 뉴스 크롤링
    def subCrawl(self, soup):
        # 모든 헤드라인 뉴스 저장
        articles = soup.find_all('div', {'class': 'cluster_group _cluster_content'})
        
        for i in range(len(articles)):
            # 각 뉴스 본문에 있는 이미지와 이미지URL를 저장할 리스트
            images=[]
            imagesURL=[]
            
            # 본문 링크
            temp = articles[i].find_all('div', {'class': 'cluster_text'})[0]
            #print(temp.a.text)         
            conURL = temp.a.get('href')
            html2 = session.get(conURL,headers=headers).content
            soup2 = BeautifulSoup(html2, 'html.parser')
            
            # 기사 제목, 내용 추출
            # 기사 요약 부분있으면 본문에서 제외하기
            summary = soup2.find('strong', {'class':'media_end_summary'})
            if summary==None:
                summary=""
            else:
                summary=summary.text
            
            contents = soup2.find('div', id= "articleBodyContents").text.replace("\n"," ").replace(str(summary),"").replace("\t"," ").replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}"," ")
            title=soup2.find('h3',id="articleTitle").text
            
            # 기사 본문이 10문장이하라면 저장하지 않는다.
            if(len(kss.split_sentences(contents)) <= 10):
                continue;
            
            # 이미지 추출
            images=soup2.find_all('span', {'class','end_photo_org'})

            for i in range(len(images)):
                imagesURL.append(images[i].find("img")["src"])
                #print(imagesURL[i])
            
            # 기사 제목과 내용, 이미지URL 각 리스트에 저장
            self.titleList.append(title)
            self.contentsList.append(contents)
            self.imageList.append(imagesURL)
        
        # 파일 저장
        self.saveNewsToFile()
    
    # 파일에 저장(DB에 저장)
    def saveNewsToFile(self):
        f = open('result.txt', 'w', encoding='utf-8')
        for i in range(len(self.titleList)):
            f.write(self.titleList[i] + "\n")
            f.writelines(self.contentsList[i]+"\n")
            f.writelines(self.imageList[i])
            f.writelines("\n\n")
        f.close()

session = requests.Session()

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
}

#driver = webdriver.Chrome(r"C:\Users\LCM\Downloads\chromedriver_win32\chromedriver.exe")
driver = webdriver.Chrome(r"C:\Users\seenw\Downloads\chromedriver_win32\chromedriver.exe")

c=newsCrawler()
c.mainCrawl()

In [9]:
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

import kss


class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.okt = Okt()
        # 불용어 불러오기
        self.stopwords = [line.rstrip('\n') for line in open('stopwords_korean3.txt', encoding = 'utf-8')]
    
    def text2sentences(self, text):
        #sentences = self.kkma.sentences(text)\
        sentences = kss.split_sentences(text)
        sentences = sentences[0:len(sentences)-2]
        
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
                
        return sentences
    
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence != '':
                nouns.append(' '.join([noun for noun in self.okt.nouns(str(sentence))
                    if noun not in self.stopwords and len(noun) > 1]))
                
        return nouns


class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
        
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}
    
    
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
                
            A[:, id] *= -d
            A[id, id] = 1
            
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

    
    
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
            
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
        #print(self.nouns)
        
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        
        return summary
    
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
            
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
            
        return keywords

In [10]:
f = open('result.txt', 'r', encoding='utf-8')

summary = []
count=1

while True:
    line = f.readline()
    # text파일에서 본문만 읽음
    if count%4 == 2:
        sum = TextRank(line)
        #print(sum.summarize(8)) # 길이가 5인 리스트 출력
        #print(len(sum.summarize(8)))
        for row in sum.summarize(6):
            summary.append(row)
            print(row)
        print()
        print()
        
    if not line:
        break
    
    count+=1

f.close()

증권사 영업점에 걸린 SK바이오사이언스 일반공모 안내 배너 / 사진=한국투자증권 제공올해 상반기 기업공개(IPO) '대어'로 꼽히는 SK바이오사이언스의 일반 청약 일정이 오늘(9일) 개시됐습니다.
금융투자업계에 따르면 SK바이오사이언스는 이날부터 내일(10일)까지 개인 투자자를 대상으로 일반 공모주 청약을 접수합니다.
공모가는 희망 공모가 최상단인 6만5천원으로 결정됐습니다.
앞서 지난 4~5일 기관 투자자 대상 수요예측에서 1천275.47대 1의 경쟁률을 기록했습니다.
기관 수요예측 경쟁률이 높았던 SK바이오팜은 상장 첫날 공모가의 두 배에 시초가를 형성한 뒤 상한가까지 직행(따상)했고, 이후 이틀 더 가격제한폭까지 치솟았습니다.
SK바이오사이언스는 코로나19 백신 개발과 위탁 생산업체를 하고 있습니다.


권오현 고문, 퇴직금만 93억원…주요 경영진 연봉 2배로 올라김기남 삼성전자 부회장[연합뉴스 자료사진]     (서울=연합뉴스) 김철선 기자 = 코로나19 팬데믹 속에서도 지난해 36조원 흑자를 이끈 삼성전자의 주요 경영진 연봉이 2배 이상으로 늘어난 것으로 나타났다.
현업에서 떠난 권오현 고문은 퇴직금을 포함해 총 172억원을 받아 보수가 가장 많았다.
김기남 부회장의 상여금은 2019년 19억5천900만원에서 지난해 66억1천200만원으로 3배 이상으로 뛰었다.
김현석 사장의 2019년 보수는 25억8천만원이다.
권오현 상임고문[연합뉴스 자료사진]    부회장직을 끝으로 현업에서 떠난 권오현 고문은 지난해 퇴직금 등으로 총 172억원을 받아 삼성전자에서 가장 많은 보수를 받았다.
지난해 삼성전자 임원 보수 순위는 권오현 고문이 가장 많았고, 그 뒤로 윤부근 고문, 신종균 고문, 전동수 전 고문, 김기남 부회장 순이었다.


이날 코스피는 전 거래일보다 30.15포인트(1.00%) 내린 2,996.11에, 원/달러 환율은 7.1원 오른 1,133.2원에 장을 마쳤다.
2021.3.8 jjaeck9@yna.co.kr(서울=연합뉴스) 이지헌 기자 = 미국 

In [13]:
pip install kss

Note: you may need to restart the kernel to use updated packages.


In [12]:
from konlpy.tag import Kkma
from konlpy.tag import Okt
from konlpy.tag import Hannanum
import kss
text='''이어 관련 신고도 들어온 적이 없었고 협회에 가서 대질 신문을 받은 적도 없다며 청원인의 주장을 반박했습니다.법적 대응 여부에 대해서는 해야 할 사항이면 할 것이라면서도, 현재 국회의원 신분인 만큼 고민하고 있다고 설명했습니다.억울함을 증언해줄 지인이 있느냐는 질문에는 제자들이 울면서 증언하겠다고 연락해오고 있다고 말했습니다.'''
text2='''한국토지주택공사(LH) 직원들의 땅 투기 의혹이 제기된 경기도 시흥시 과림동 일대. 채혜선 기자           한국토지주택공사(LH) 직원들의 경기 광명과 시흥 지역 사전 투기 의혹이 짙어지고 있다. 100억원대에 이르는 땅값의 60%인 58억원가량을 북시흥농협 한 곳에서 대출받은 것으로 드러났기 때문이다.       북시흥농협은 2금융권인 단위농협(상호금융)으로 이곳의 금리는 은행권보다 상대적으로 높다. 비싼 금리에도 대출 문턱이 낮은 탓에 한도를 최대한 끌어올 수 있는 단위농협의 문을 두드렸을 가능성이 크다는 해석이 나오는 이유다.       특히 시중은행에서는 흔치 않은 ‘대출 쪼개기’도 있었다. 여럿이 모여 공동 소유할 땅을 담보로 각각 돈을 빌려 투자금을 마련하는 방식으로 이들이 단위농협을 택한 또 다른 이유로도 풀이된다.      중앙일보가 해당 거래의 등기부 등본을 살펴본 결과 LH 직원들은 2018년부터 3년간 경기도 시흥시 과림동과 무지내동 일원에서 99억원어치(약 2만㎡) 농지를 매입했다. 투자금의 63%(약 62억5800만원) 정도를 금융사에서 빌렸다. 일반적으로 토지는 대출액의 120%를 근저당권 채권최고액(전체 75억1300만원)으로 잡는 점을 고려해 역산한 추정치다.       이 중 4억원 가량(농협은행)을 제외한 58억5800만원을 대출받은 곳이 경기도 시흥의 북시흥농협이다. 공교롭게도 대출을 받은 LH 직원 10명(이 중 3명은 중복 대출)이 줄줄이 한 곳의 단위농협을 방문해 대출 신청서를 작성했다는 얘기다.     LH 직원 투기 의혹 관련 대출 현황. 그래픽=김은교 kim.eungyo@joongang.co.kr      ━   상호금융, LTV 최대 80%까지 가능      은행업계의 분석은 대출 한도를 높이기 위한 ‘꼼수’라는 데 무게가 실린다. 우선 단위농협이 속한 상호금융은 주택처럼 땅을 담보로 돈을 빌릴 때 인정되는 주택담보인정비율(LTV)이 시중은행보다 높은 편이다. 현재 금융당국은 행정지도를 통해 상호금융의 비주택 LTV를 40~70% 범위로 제한했다. 담보물의 몸값(회수 가치)이 높으면 최대 10%포인트까지 가산은 허용한다.       반면 시중은행에서 개인의 비주택 LTV는 평균 60% 수준이 적용된다. 비주택은 아파트처럼 일률적인 지역별 LTV 규제를 적용하지는 않지만 까다로운 대출심사를 거쳐야 한다. 더욱이 최근 시중은행은 대출자의 전체 소득에서 빚(원리금)을 갚을 능력을 보는 총부채원리금상환비율(DSR)도 따지고 있어 돈을 빌리기가 더 어렵다.         시중은행 관계자는 “(약 3%대인 시중은행의 대출금리보다) 1~2%포인트 더 비싼 이자를 고려하고도 상호금융을 찾았다는 것은 DSR 같은 규제를 받지 않고 최대한 대출 한도를 높이기 위한 것으로 보인다”고 말했다.       땅의 몸값도 더 높게 평가받기 위해 단위농협을 택했다는 시각도 있다. 상호금융 관계자는 “단위농협 특성상 지역밀착형으로 운영되다 보니 (매입하는 땅의) 인근 지점을 이용하는 게 땅값을 조금이라도 높게 책정하는 데 유리할 수 있다”고 했다.       ━   5명이 ‘대출 쪼개기’로 17억 투자금 마련      지분 쪼개기로 대출 받은 LH 직원들. 그래픽=김은교 kim.eungyo@joongang.co.kr           투기 의혹을 키우는 것은 직원끼리 대출 부담을 나눠 투자금을 마련한, 이른바 ‘대출 쪼개기’다.       등기부 등본에 따르면 경기도 시흥시 과림동 일대의 15억1000만원 상당(3996㎡)의 농지 소유주는 총 4명의 LH 직원이다. 이 중 3명이 농지를 담보로 각각 대출을 받았다. 전체 대출금 추정치는 9억5000만원(근저당권 채권최고액 11억4400만원)이다.       땅값이 비쌀수록 공동소유자를 늘려 대출을 쪼갰다. 22억원 상당의 필지는 5명이 각각 2억~4억원가량씩 대출을 받아 17억원 가량을 모았다. 땅값의 77%를 대출로 마련한 것이다. 투자금 부담이 줄면서 LH 직원 3명은 2~3개의 필지의 공동소유주로 이름을 올렸다.       상호금융권의 대출 문턱이 상대적으로 낮은 데다 심사의 허술한 점을 노렸다는 것이다. 익명을 요구한 시중은행 대출심사 전문가는 “시중은행에서 공동소유자가 여럿인 땅을 담보로 소유자 각자에게 대출해주는 일은 흔치 않다”고 말했다. 특히 “차주가 여럿인 땅은 한 명이라도 이자를 연체하는 등 문제가 생기면 부실을 관리하는 게 쉽지 않아 대출 심사를 통과하기 어렵다”고 덧붙였다.         그렇다면 LH 직원이 농민(조합원)이 이용하는 단위농협에서 대출받을 수 있을까. 조합원이 아니어도 일반인도 지역농협에서 대출은 할 수 있다. 다만 농지를 담보로 대출을 받으려면 농지법상 농업경영계획서를 내고, 행정기관의 승인을 받아야 한다는 게 농협중앙회의 설명이다.       ━   농협 “사회적 문제된 만큼 현장 조사 진행 중”        LH 직원들이 조직적으로 사들였다는 의혹이 제기된 경기도 시흥시 과림동 현장에 묘목이 식재돼 있다. 오종택 기자           농협중앙회 측은 “LTV가 70%인데 (문제가 된) 모든 대출이 70% 이하로 이뤄진 것으로 확인됐다”며 대출 과정에서 위법사항은 없었다고 강조했다. 중앙회측은 다만 "사회적으로 문제가 되고 있는 만큼 해당 건에 대해 현장 조사를 진행하고 있다"고 밝혔다.       농협중앙회의 고위 관계자는 “한 명이 먼저 대출을 받았는데 (대출이) 잘 나온다고 입소문이 나면서 몰린 거 같다”며 “3년에 걸쳐 10여명이 차례로 대출을 받은 것이지, 한꺼번에 몰린 것은 아니다”고 말했다.      현재 금융당국은 LH 직원의 땅 투기 의혹 관련해 대출과정에 문제가 없었는지 살펴보고 있다. 금융위원회 관계자는 “현재 사실관계를 확인 중”이라며 “상호금융은 LTV 규제가 행정지도 사항이라 위반을 했더라도 제재는 어렵다”고 했다.      이민환 인하대 글로벌금융학과 교수는 “상호금융이 본연의 취지와 다르게 담보대출 위주로 영업하며 돈놀이를 하고 있다”며 “토지 담보가 있으면 심사를 거의 안 하다시피 해서 대출을 내주는 관행이 단위 농협 등 대부분의 상호금융에 퍼져있다”고 말했다.      염지현ㆍ안효성ㆍ윤상언 기자 yjh@joonang.co.kr   ▶ 네이버 구독 첫 500만 중앙일보 받아보세요▶ 넌 뉴스를 찾아봐? 난 뉴스가 찾아와!▶ 당신이 궁금한 코로나, 여기 다 있습니다ⓒ중앙일보(https://joongang.co.kr), 무단 전재 및 재배포 금지   '''

kkma = Kkma()
ss = kkma.sentences(text)
print(ss)

#hannanum = Hannanum()
#hannanum.sentences(text)

for sent in kss.split_sentences(text2):
    print(sent)
    print()

#sss = kss.split_sentences(text2)
#print(sss)




['이어 관련 신고도 들어온 적이 없었고 협회에 가서 대질 신문을 받은 적도 없다며 청원인의 주장을 반박했습니다.', '법적 대응 여부에 대해서는 해야 할 사항이면 할 것이라면서도, 현재 국회의원 신분인 만큼 고민하고 있다고', '설명했습니다.', '억울함을 증언해 줄 지인이 있느냐', '는 질문에는 제자들이 울면서 증언하겠다고', '연락해 오고 있다고', '말했습니다.']
한국토지주택공사(LH) 직원들의 땅 투기 의혹이 제기된 경기도 시흥시 과림동 일대. 채혜선 기자           한국토지주택공사(LH) 직원들의 경기 광명과 시흥 지역 사전 투기 의혹이 짙어지고 있다.

100억원대에 이르는 땅값의 60%인 58억원가량을 북시흥농협 한 곳에서 대출받은 것으로 드러났기 때문이다.

북시흥농협은 2금융권인 단위농협(상호금융)으로 이곳의 금리는 은행권보다 상대적으로 높다.

비싼 금리에도 대출 문턱이 낮은 탓에 한도를 최대한 끌어올 수 있는 단위농협의 문을 두드렸을 가능성이 크다는 해석이 나오는 이유다.

특히 시중은행에서는 흔치 않은 ‘대출 쪼개기’도 있었다.

여럿이 모여 공동 소유할 땅을 담보로 각각 돈을 빌려 투자금을 마련하는 방식으로 이들이 단위농협을 택한 또 다른 이유로도 풀이된다.

중앙일보가 해당 거래의 등기부 등본을 살펴본 결과 LH 직원들은 2018년부터 3년간 경기도 시흥시 과림동과 무지내동 일원에서 99억원어치(약 2만㎡) 농지를 매입했다.

투자금의 63%(약 62억5800만원) 정도를 금융사에서 빌렸다.

일반적으로 토지는 대출액의 120%를 근저당권 채권최고액(전체 75억1300만원)으로 잡는 점을 고려해 역산한 추정치다.

이 중 4억원 가량(농협은행)을 제외한 58억5800만원을 대출받은 곳이 경기도 시흥의 북시흥농협이다.

공교롭게도 대출을 받은 LH 직원 10명(이 중 3명은 중복 대출)이 줄줄이 한 곳의 단위농협을 방문해 대출 신청서를 작성했다는 얘기다.

LH 직원 투기 의혹 관련 대출 현황. 그래픽=김은교 kim.eung